# Predicting response times for StackOverflow questions
Aaron Pereira and Ryan Becwar</center>

## Introduction

The main aim of this study is to try to analyze the MSR2019 SOTorrent dataset to try to gain some insight about stackoverflow data and try to predict the amount of time a question would take to find an accepted answer. We tried to make this prediction based on various factors based on the properties of the user asking the question and the question itself. We tried to list down some features based on these properties that we could identify and tried feeding them into different machine learning models and tried to find some correlation between these features and the response time

### Initial problems related to dataset usage

- The MSR-2019 page linked to BigQuery but the link seemed to be dead
- We found an alernate BigQuery link but we had to make a google account to access that
- Free/Trial accounts had a 1TB processing limit and our queries processed more data than that

We then decided to go for the alternate link on the 'empirical-software.engineering' to access the dataset on Zenodo.com. We faced the following problems with this source

- Zenodo hosted individual files for all the tables. We couldn't run interactive queries like BigQuery
- Majority of our processing involved the 'Posts' table that came up to 70 GB once extracted
- The data was available in XML format which needed additional processing overhead
- the 'Posts' table also involved a join on itself which would be computationally challenging given its size
- We also needed to process a few more tables to get all the data we needed

We used a variety of pre-processing steps to get the data in a format that we deemed fit to run analysis on that will be explained in detail in the following setions

### Description of the dataset

The dataset we planned to use was structured in the form of XML files. Each table joined others using foreign keys that were a little complicated to understand sometimes. The following is a pictorial representation of the dataset:

<img src="https://empirical-software.engineering/assets/images/sotorrent_2018-12-09_model.png" style="text-align:center"></img>
<center>source : https://empirical-software.engineering</center>
    
We wanted to center our research around 'Tags' and how they affect the ability of a question to recieve an accepted answer. We decided to use the following Tables for our research:

- Posts
- Tags
- Users
- Postlinks

reputation,Votes,Downvotes,LinkType,TimeToAnswer,Question Body Char Count,Question Body Word Count,Question Title Word Count,Question Title Word Count,Question creation date,Title end in ?,Question creation datee

We planned to extract the following fields from the tables mentioned above

### Posts
- Question character count
- Question word count
- Does the question end with a '?'
- __Time taken to get an accepted answer (Question creation date - Accepted answer creation date)__

### Users
- Reputation
- Total upvotes
- Total downvotes
- Number of badges the user has

### Postlinks
- type of links the questions have

### Tags
Tags seemed like the most important criteria and there were around 60000 unique tags present in the dataset. We needed to find an appropriate method to represent these tags in the predictions. For that, we followed the following procedure

- Filter questions based on a programming language (extract only those questions that had 'C++' in the question, body or tags)
- Extract all the tags from these questions
- Rank the Tags based on the frequency of occurance
- Shortlist the top 50 Tags
- Have each tag represented as a binary feature

# Data Pre-Processing

We first downloaded each data table from Zenodo. These tables were in the form of 7z archive files. We then extracted each one of these tables. They totalled to 110 GB of data on the whole.

## Processing large XML files
Consider 'Posts.xml'. This was a single XML file. No library would be able to process such a large XML object. So we split this file into smaller files using the following linux command:

In [ ]:
split -l50000 Comments.xml --verbose

This command splits the big file into smaller files having 50000 lines each. But this would now not be valid XML. We used the following script to convert all these split files into valid XML:

In [ ]:
#!/bin/bash
for filename in /media/aaron/1b564795-7979-4d3d-8f05-635dda73c1e4/cs515/*; do
	sed -i '1i<?xml version="1.0" encoding="utf-8"?><posts>' ${filename};
	echo "</posts>" >> $filename;
	#sed -i '1d' $filename
done

This now converted the multiple split files into valid XML. We then ran the following python code snippet to get a CSV file of only the required features from each XML. Its important to note that we now divided the 'Posts' data into 'questions' and 'answers' based on 'PostType'. We also restricted the questions based on only those that contained 'C++'. The following is the code snippet we used for generating a CSV for 'Questions'. We used similar snippits to generate other CSVs too.

There are a set of Lists created at the start. These Lists contain all the features that we needed to extract for all the tables. We ran a similar snippet for all the tables that we needed.

In [ ]:
import xml.etree.ElementTree as ET
import csv
import os
commentsList = ['Id','CreationDate']
userlist = ['UserId','Class','CreationDate','LastAccessDate','Views','UpVotes','DownVotes','AccountId']
badgelist = ['UserId','Class']
tagslist = ['Id','TagName']
postLinklist = ['Id','PostId','LinkTypeId']
questionslist = ['Id','AcceptedAnswerId','CreationDate','OwnerUserId', 'Tags', 'Title', 'Body']
answerlist = ['Id','ParentId','CreationDate','OwnerUserId']
extra = ['Title','Body']
table = "questions"

#filebasepath = "/home/aaron/CS-515Project/"
filebasepath = "/media/aaron/1b564795-7979-4d3d-8f05-635dda73c1e4/"

with open(table+'.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    count = 0
    for filename in os.listdir(filebasepath+table+"/"):
        try:
            tree = ET.parse(filebasepath+table+'/'+filename)
            root = tree.getroot()
            for elem in root:
                writeContent = []
                postType = int(elem.attrib['PostTypeId'])
                if postType == 1 and "c++" in elem.attrib['Tags']:
                    for attrib in elem.attrib:
                        if attrib in questionslist:
                            if attrib not in extra:
                                writeContent.append(elem.attrib[attrib]);
                            else:
                                if attrib == "Title":
                                    if str(elem.attrib[attrib]).endswith("?"):
                                        writeContent.append(0)
                                        writeContent.append(elem.attrib[attrib].replace(",","_"))
                                        writeContent.append(len(elem.attrib[attrib]))
                                        writeContent.append(len(elem.attrib[attrib].split(" ")))
                                    else:
                                        writeContent.append(1);
                                        writeContent.append(elem.attrib[attrib].replace(",","_"))
                                        writeContent.append(len(elem.attrib[attrib]))
                                        writeContent.append(len(elem.attrib[attrib].split(" ")))
                                if attrib == "Body":
                                    writeContent.append(len(elem.attrib[attrib]))
                                    writeContent.append(len(elem.attrib[attrib].split(" ")))


                    if len(writeContent)==len(questionslist)+4:
                        writer.writerow(writeContent)
        except:
            print(filename)


## Inserting data into MySql

The code above generated a bunch of CSV files for us. We now decided that it would be best to place these files into a MySql database and perform a join to get the data we needed. We used the following command to load this data into our MySql database.

In [ ]:
LOAD DATA INFILE '/var/lib/mysql-files/questions.csv' 
INTO TABLE questions 
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
IGNORE 0 ROWS;

The above command was executed for every CSV file and we made a corresponding table for each CSV file. Our total data was now reduced to 1.3 GB. We then ran the following query to extract the final dataset and saved the results as a CSV file.

In [ ]:
select Tags, users.Reputation as 'reputation',
				users.Upvotes as 'Votes', 
				users.DownVotes as 'Downvotes', 
                postlinks.LinkTypeId as 'LinkType',  
                TIMESTAMPDIFF(MINUTE,questions.CreationDate,answers.CreationDate) as 'TimeToAnswer',
                questions.BodyCharCount as 'Question Body Char Count',
                questions.BodyWordCount as 'Question Body Word Count',
                questions.TitleCharCount as 'Question Title Char Count',
                questions.TitleWordCount as 'Question Title Word Count',
                questions.CreationDate as 'Question creation date', 
                questions.TitleQuestionMark as 'Title end in ?',
                answers.CreationDate as 'Question creation date' 
                from questions 
						left join answers on questions.AcceptedAnswerId = answers.Id 
						left join users on questions.OwnerUserId = users.Id 
                        left join postlinks on questions.Id = PostId;

## Generating Tag Features

The last step was to process extract the top 50 tags and represent them as one feature column each. First we needed to extract the 50 most common tags. We extracted the list of all tags using the following query

In [ ]:
select tags from questions;

We now cleaned the data a little as each tag was enclosed in '< >'. We cleaned these tags and placed them in an online word frequency generator. the most common tags that were observed were he following:

"c++","c++11","qt","templates","visual-c++","boost","arrays","pointers","vector","windows","winapi","stl","multithreading","string","opencv","linux","class","opengl","gcc","visual-studio","algorithm","inheritance","visual-studio-2010","function","mfc","java","c++14","g++","dll","python","operator-overloading","struct","c++-cli","oop	3307","memory-management","iterator","performance","cmake","memory","linker","sockets","reference","compiler-errors","language-lawyer","macos","const","std","lambda","file","debugging"

We now used the following the following code snippet to generate tag based features.


In [ ]:
import xml.etree.ElementTree as ET
import csv
import pandas as pd

import csv

tagList = ["c++","c++11","qt","templates","visual-c++","boost","arrays","pointers","vector","windows","winapi","stl","multithreading","string","opencv","linux","class","opengl","gcc","visual-studio","algorithm","inheritance","visual-studio-2010","function","mfc","java","c++14","g++","dll","python","operator-overloading","struct","c++-cli","oop	3307","memory-management","iterator","performance","cmake","memory","linker","sockets","reference","compiler-errors","language-lawyer","macos","const","std","lambda","file","debugging"]

frame = pd.read_csv('newFull.csv')

with open("encodingFull.csv", 'a', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    first = True
    with open('newFull.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            if first:
                writer.writerow(tagList + row[1:len(row)])
                first = False
            else:
                rowsAfterTags = row[1:len(row)]
                tags = row[0][1:-1].split("><")
                tagFeatures = []
                for tag in tagList:
                    if tag in tags:
                        tagFeatures.append(1)
                    else:
                        tagFeatures.append(0)
                totalFeatureRow = tagFeatures + rowsAfterTags
                writer.writerow(totalFeatureRow)


This finally generated the final dataset in the form of a CSV file that we can then use to perform data analysis. Our final dataset has a total of 367k entries.